# Composite Bloq
`CompositeBloq` is our primary container class for writing quantum programs. 

The `Bloq` interface describes a black-box quantum operation that guarantees certain input and output registers and can be annotated with known quantities (like resource counts). A special type of `Bloq` is `CompositeBloq`. Instead of having its own name, attributes, resource counts, and others, it is a container class that is simply a collection of sub-bloqs. Specifically, it encodes a graph where we not only include the sub-bloqs but which outputs are connected to which inputs. 

In [ ]:
from typing import *
import numpy as np

## Bloq Builder
Let's see how we can take an example `Bloq`, create two instances of them, and wire them up in two different ways.

In [ ]:
from cirq_qubitization.quantum_graph.bloq import Bloq
from cirq_qubitization.quantum_graph.composite_bloq import CompositeBloqBuilder, SoquetT
from cirq_qubitization.quantum_graph.fancy_registers import FancyRegisters
from cirq_qubitization.jupyter_tools import show_bloq

# An example Bloq:
from cirq_qubitization.quantum_graph.bloq_test import TestCNOT
bloq = TestCNOT()

# Wire up (way 1)
bb = CompositeBloqBuilder()
q0 = bb.add_register('q0', 1)
q1 = bb.add_register('q1', 1)
q0, q1 = bb.add(bloq, control=q0, target=q1)
q0, q1 = bb.add(bloq, control=q0, target=q1)
cbloq = bb.finalize(q0=q0, q1=q1)
show_bloq(cbloq)

In [ ]:
# Wire up (way 2)
bb = CompositeBloqBuilder()
q0 = bb.add_register('q0', 1)
q1 = bb.add_register('q1', 1)
q0, q1 = bb.add(bloq, control=q0, target=q1)
q0, q1 = bb.add(bloq, control=q1, target=q0) ## !!
cbloq = bb.finalize(q0=q0, q1=q1)
show_bloq(cbloq)

## Decompose Bloq

You can direcly contruct composite bloqs using `CompositeBloqBuilder` per above. The other main use of composite bloqs is the return type of `Bloq.decompose_bloq()`. When defining a bloq, you can provide its decomposition by overriding the `build_composite_bloq` method. In this case, the bloq builder and registers are set up for you and you just need to add the operations.

In [ ]:
class TestTwoCNOT(Bloq):
    @property
    def registers(self) -> FancyRegisters:
        return FancyRegisters.build(q1=1, q2=1)

    def build_composite_bloq(
        self, bb: 'CompositeBloqBuilder', q1: 'Soquet', q2: 'Soquet'
    ) -> Dict[str, SoquetT]:
        q1, q2 = bb.add(TestCNOT(), control=q1, target=q2)
        q1, q2 = bb.add(TestCNOT(), control=q2, target=q1)
        return {'q1': q1, 'q2': q2}

In [ ]:
show_bloq(TestTwoCNOT())

In [ ]:
show_bloq(TestTwoCNOT().decompose_bloq())

## Debug Text

The graph structure is most easily viewed as a diagram, but composite bloqs also expose a textual description where each sub-bloq instance is printed in topologically-sorted order. Below each subbloq, the incoming (left) and outgoing (right) connections are printed.

In [ ]:
print(cbloq.debug_text())

## Copy

We can perform a copy of a composite bloq, which will produce a new composite bloq whose bloq instances are different. This is an incredibly uninteresting operation in the abstract because CompositeBloqs are immutable. However, you can inspect the `copy` code to see how it forms the basis for more interesting copy-with-modification methods discussed later.

In [ ]:
from cirq_qubitization.quantum_graph.composite_bloq_test import Atom, TestSerialBloq, TestParallelBloq

cbloq = TestParallelBloq().decompose_bloq()
cbloq2 = cbloq.copy()

# They're the same!
display(show_bloq(cbloq))
display(show_bloq(cbloq2))

To try to show that something is actually happening, we use the following monkey-patching code to override `BloqBuilder`'s internal counter for numbering bloq instances so the copied version has different indices.

In [ ]:
from contextlib import contextmanager

@contextmanager
def hacked_bb_init():
    # monkey-patch BloqBuilder to offset the bloq instance counter.
    
    old_bb_init_method = CompositeBloqBuilder.__init__
    
    def _new_init(self, *args, **kwargs):
        old_bb_init_method(self, *args, **kwargs)
        self._i = 100

    try:
        CompositeBloqBuilder.__init__ = _new_init
        yield
    finally:
        CompositeBloqBuilder.__init__ = old_bb_init_method

Now when we iterate through the original cbloq's connections and the copy's, we see that the connectivity is the same but the bloq instance indices are different.

In [ ]:
cbloq = TestParallelBloq().decompose_bloq()

with hacked_bb_init():
    cbloq2 = cbloq.copy()

for cxn1, cxn2 in zip(cbloq.connections, cbloq2.connections):
    print(cxn1)
    print(cxn2)
    print()

## Add from

We can use `bb.add_from` to add the contents of a composite bloq wholesale to the current bloq-under-construction. This has the effect of flattening one level of structure during bloq construction. In the following cells, we connect `TestParallelBloq`s serially but vary when we call `bb.add` vs `bb.add_from`.

In [ ]:
# Just call add
bb = CompositeBloqBuilder()
stuff = bb.add_register('stuff', 3)
stuff, = bb.add(TestParallelBloq(), stuff=stuff)
stuff, = bb.add(TestParallelBloq(), stuff=stuff)
bloq = bb.finalize(stuff=stuff)
show_bloq(bloq)

In [ ]:
# `add_from` on second one
bb = CompositeBloqBuilder()
stuff = bb.add_register('stuff', 3)
stuff, = bb.add(TestParallelBloq(), stuff=stuff)
stuff, = bb.add_from(TestParallelBloq(), stuff=stuff)
bloq = bb.finalize(stuff=stuff)

show_bloq(bloq)

In [ ]:
# `add_from` on first one
bb = CompositeBloqBuilder()
stuff = bb.add_register('stuff', 3)
stuff, = bb.add_from(TestParallelBloq(), stuff=stuff)
stuff, = bb.add(TestParallelBloq(), stuff=stuff)
bloq = bb.finalize(stuff=stuff)

show_bloq(bloq)

In [ ]:
# `add_from` on middle one
bb = CompositeBloqBuilder()
stuff = bb.add_register('stuff', 3)
stuff, = bb.add(TestParallelBloq(), stuff=stuff)
stuff, = bb.add_from(TestParallelBloq().decompose_bloq(), stuff=stuff)
stuff, = bb.add(TestParallelBloq(), stuff=stuff)

bloq = bb.finalize(stuff=stuff)
show_bloq(bloq)

## Controlled

`ControlledBloq(subbloq)` represents a controlled version of `subbloq`. Its decompose method will call `subbloq`'s decompose and wrap each of the child bloqs in `ControlledBloq`.

In [ ]:
from cirq_qubitization.quantum_graph.meta_bloq import ControlledBloq

bloq = ControlledBloq(subbloq=Atom())
show_bloq(bloq)

### Controlled Serial Bloq

In [ ]:
bloq = ControlledBloq(subbloq=TestSerialBloq())
display(show_bloq(bloq))
display(show_bloq(bloq.decompose_bloq()))


### Controlled Parallel Bloq

In [ ]:
bloq = ControlledBloq(subbloq=TestParallelBloq())
display(show_bloq(bloq))
display(show_bloq(bloq.decompose_bloq()))